# Use Spark for Scala to load data and run SQL queries
This notebook introduces basic Spark concepts and helps you to start using Spark for Scala.

Some familiarity with Scala is recommended.

In this notebook, you'll use the publicly available **mtcars** data set from *Motor Trend* magazine to learn some basic Scala. You'll learn how to load data, create a Spark DataFrame, aggregate data, run mathematical formulas, and run SQL queries against the data.


## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Initialize an SQLContext](#Initialize_an_SQLContext)
3. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
4. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
5. [Operate on columns](#Operate_on_columns)
6. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The local Scala DataFrame called **mtcars** includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`

In [4]:
import sys.process._
import java.net.URL
import java.io.File

def fileDownloader(url: String, filename: String) = {
    new URL(url) #> new File(filename) !!
}

fileDownloader("https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv", "mtcars.csv")

""

<a id='Initialize_an_SQLContext'></a>
## 2. Initialize an SQLContext
To work with a DataFrame, you need an SQLContext. You create this SQLContext by using `SQLContext(sc)`. 
A SparkContext named sc, which has been created for you, is used to initialize the SQLContext:

In [5]:
import au.com.bytecode.opencsv.CSVParser
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

<a id='Create_a_Spark_DataFrame'></a>
## 3. Create a Spark DataFrame
Using the SQLContext and the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [6]:
// Define the schema using a case class.
case class Cars(car: String, mpg: String, cyl: String, disp: String, hp: String, drat: String,
 wt: String, qsec: String, vs: String, am: String, gear: String, carb: String);

In [10]:
val csv = sc.textFile("mtcars.csv")
val headerAndRows = csv.map(line => line.split(",").map(_.trim))
val header = headerAndRows.first
val data = headerAndRows.filter(_(0) != header(0))
val mtcars = data.map(p => Cars(p(0), p(1), p(2), p(3), p(4), p(5), p(6), p(7), p(8), p(9), p(10), p(11))).toDF()
mtcars.printSchema

root
 |-- car: string (nullable = true)
 |-- mpg: string (nullable = true)
 |-- cyl: string (nullable = true)
 |-- disp: string (nullable = true)
 |-- hp: string (nullable = true)
 |-- drat: string (nullable = true)
 |-- wt: string (nullable = true)
 |-- qsec: string (nullable = true)
 |-- vs: string (nullable = true)
 |-- am: string (nullable = true)
 |-- gear: string (nullable = true)
 |-- carb: string (nullable = true)



In [13]:
headerAndRows.take(5)

Array(Array(car, mpg, cyl, disp, hp, drat, wt, qsec, vs, am, gear, carb), Array(Mazda RX4, 21, 6, 160, 110, 3.9, 2.62, 16.46, 0, 1, 4, 4), Array(Mazda RX4 Wag, 21, 6, 160, 110, 3.9, 2.875, 17.02, 0, 1, 4, 4), Array(Datsun 710, 22.8, 4, 108, 93, 3.85, 2.32, 18.61, 1, 1, 4, 1), Array(Hornet 4 Drive, 21.4, 6, 258, 110, 3.08, 3.215, 19.44, 1, 0, 3, 1))

## ***** Using SparkSession - A Unified Entry Point in Apache Spark 2.0 *****

In [1]:
val dataFrame_cars = spark.read.format("CSV").option("header","true").load("mtcars.csv")

In [4]:
dataFrame_cars.show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|                car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|          Mazda RX4|  21|  6|  160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|      Mazda RX4 Wag|  21|  6|  160|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|         Datsun 710|22.8|  4|  108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|     Hornet 4 Drive|21.4|  6|  258|110|3.08|3.215|19.44|  1|  0|   3|   1|
|  Hornet Sportabout|18.7|  8|  360|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|            Valiant|18.1|  6|  225|105|2.76| 3.46|20.22|  1|  0|   3|   1|
|         Duster 360|14.3|  8|  360|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19|   20|  1|  0|   4|   2|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|           Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|
|          M

Display the content of the Spark DataFrame:

In [9]:
dataFrame_cars

[car: string, mpg: string ... 10 more fields]

In [7]:
mtcars

[car: string, mpg: string ... 10 more fields]

In [ ]:
mtcars.show(32)

Try different ways of retrieving subsets of the data. For example, get the first 5 values in the **mpg** column:

In [ ]:
mtcars.select("mpg").show(5)

Filter the DataFrame to retain only rows with **mpg** values that are less than 18:

In [ ]:
mtcars.filter(mtcars("mpg") < 18).show()

<a id='Aggregate_data_after_grouping_by_columns'></a>
## 4. Aggregate data after grouping by columns
Spark DataFrames support a number of common functions to aggregate data after grouping. For example, you can compute the average weight of cars as a function of the number of cylinders:

In [2]:
import org.apache.spark.sql.functions._
mtcars.groupBy("cyl").agg(avg("wt")).show()

+---+-----------------+
|cyl|          avg(wt)|
+---+-----------------+
|  8|3.999214285714286|
|  6|3.117142857142857|
|  4|2.285727272727273|
+---+-----------------+



In [1]:
import org.apache.spark.sql.functions._
dataFrame_cars.groupBy("cyl").agg(avg("wt")).show()

+---+------------------+
|cyl|           avg(wt)|
+---+------------------+
|  8|3.9992142857142867|
|  6| 3.117142857142857|
|  4| 2.285727272727273|
+---+------------------+



<p>You can also sort the output from the aggregation to determine the most popular cylinder configuration in the DataFrame:</p>

In [3]:
mtcars.groupBy("cyl").agg(count("wt")).sort($"count(wt)".desc).show()

+---+---------+
|cyl|count(wt)|
+---+---------+
|  8|       14|
|  4|       11|
|  6|        7|
+---+---------+



In [4]:
dataFrame_cars.groupBy("cyl").agg(count("wt")).sort($"count(wt)".desc).show()

+---+---------+
|cyl|count(wt)|
+---+---------+
|  8|       14|
|  4|       11|
|  6|        7|
+---+---------+



<a id='Operate_on_columns'></a>
## 5. Operate on columns
Scala provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [5]:
mtcars.withColumn("wtTon", mtcars("wt") * 0.45).select(("car"),("wt"),("wtTon")).show(6)

+-----------------+-----+-------+
|              car|   wt|  wtTon|
+-----------------+-----+-------+
|        Mazda RX4| 2.62|  1.179|
|    Mazda RX4 Wag|2.875|1.29375|
|       Datsun 710| 2.32|  1.044|
|   Hornet 4 Drive|3.215|1.44675|
|Hornet Sportabout| 3.44|  1.548|
|          Valiant| 3.46|  1.557|
+-----------------+-----+-------+
only showing top 6 rows



In [7]:
dataFrame_cars.withColumn("wtTon", dataFrame_cars("wt") * 0.45).select(("car"),("wt"),("wtTon")).show(6)

+-----------------+-----+-------+
|              car|   wt|  wtTon|
+-----------------+-----+-------+
|        Mazda RX4| 2.62|  1.179|
|    Mazda RX4 Wag|2.875|1.29375|
|       Datsun 710| 2.32|  1.044|
|   Hornet 4 Drive|3.215|1.44675|
|Hornet Sportabout| 3.44|  1.548|
|          Valiant| 3.46|  1.557|
+-----------------+-----+-------+
only showing top 6 rows



<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 6. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [8]:
mtcars.registerTempTable("cars")

val highgearcars = sqlContext.sql("SELECT car, gear FROM cars WHERE gear >= 5")
highgearcars.show()

+--------------+----+
|           car|gear|
+--------------+----+
| Porsche 914-2|   5|
|  Lotus Europa|   5|
|Ford Pantera L|   5|
|  Ferrari Dino|   5|
| Maserati Bora|   5|
+--------------+----+



In [9]:
dataFrame_cars.registerTempTable("dataFrame_cars")

val dataFrame_highgearcars = sqlContext.sql("SELECT car, gear FROM dataFrame_cars WHERE gear >= 5")
dataFrame_highgearcars.show()

+--------------+----+
|           car|gear|
+--------------+----+
| Porsche 914-2|   5|
|  Lotus Europa|   5|
|Ford Pantera L|   5|
|  Ferrari Dino|   5|
| Maserati Bora|   5|
+--------------+----+



## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

### Authors

**Saeed Aghabozorgi**, PhD, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in the data mining field and an expert in developing advanced analytic methods like machine learning and statistical modelling on large data sets.

**Polong Lin** is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds an M.Sc. in Cognitive Psychology.